In [ ]:
import geopandas as gpd
import plotly.express as px
from shapely.geometry import Point, box

from srai.plotting.folium_wrapper import plot_regions
from srai.regionalizers import AdministrativeBoundaryRegionalizer, geocode_to_region_gdf

## Regionalize city

Basic usage of the `AdministrativeBoundaryRegionalizer` with a city boundary. 

Here `admin_level` equal to 9 defines city districts in Poland.

In [ ]:
wroclaw_gdf = geocode_to_region_gdf(query=["R451516"], by_osmid=True)
plot_regions(wroclaw_gdf)

In [ ]:
abr = AdministrativeBoundaryRegionalizer(admin_level=9)

In [ ]:
wro_result_gdf = abr.transform(gdf=wroclaw_gdf)
wro_result_gdf.head()

In [ ]:
plot_regions(wro_result_gdf)

## Regionalize country

How to return an empty region covering water bodies outside of the land.

Here `admin_level` equal to 4 defines country regions in Madagascar.

In [ ]:
madagascar_bbox = box(minx=43.21418, miny=-25.61143, maxx=50.48704, maxy=-11.951126)
madagascar_bbox_gdf = gpd.GeoDataFrame({"geometry": [madagascar_bbox]}, crs="EPSG:4326")

In [ ]:
abr = AdministrativeBoundaryRegionalizer(admin_level=4, return_empty_region=True)

In [ ]:
madagascar_result_gdf = abr.transform(gdf=madagascar_bbox_gdf)
madagascar_result_gdf.tail()

In [ ]:
plot_regions(madagascar_result_gdf)

## Regionalize Europe

Option to slightly increase the value of `toposiplify` to simplify geometries even more.

Here `admin_level` equal to 2 defines countries.

In [ ]:
eu_bbox = box(minx=-10.478556, miny=34.633284672291, maxx=32.097916, maxy=70.096054)
eu_bbox_gdf = gpd.GeoDataFrame({"geometry": [eu_bbox]}, crs="EPSG:4326")

In [ ]:
abr = AdministrativeBoundaryRegionalizer(admin_level=2, toposimplify=0.0005)

In [ ]:
eu_result_gdf = abr.transform(gdf=eu_bbox_gdf)

In [ ]:
eu_result_gdf.head()

In [ ]:
plot_regions(eu_result_gdf)

## Toposimplify differences

Shows differences in simplification of small regions using four values: `1e-4`, `1e-3`, `1e-2` and `0.1`. Those values are in degress, since it uses Douglas-Peucker simplification algorithm.

`1e-4` is the default value and is equal to about 11.1m accuracy.

More info: https://github.com/mattijn/topojson

Here `admin_level` equal to 6 defines city districts in Singapore.

In [ ]:
singapore_bbox = box(minx=103.5111238, miny=1.1263707, maxx=104.1313374, maxy=1.4787511)
singapore_bbox_gdf = gpd.GeoDataFrame({"geometry": [singapore_bbox]}, crs="EPSG:4326")

In [ ]:
results = {}
for value in [0.0001, 0.001, 0.01, 0.1]:
    abr = AdministrativeBoundaryRegionalizer(admin_level=6, toposimplify=value)
    results[value] = abr.transform(gdf=singapore_bbox_gdf)

In [ ]:
minx, miny, maxx, maxy = singapore_bbox.bounds
for epsilon, result in results.items():
    fig = px.choropleth_mapbox(
        result,
        geojson=result,
        color=result.index,
        locations=result.index,
        center={"lat": 1.3119350704252704, "lon": 103.82412242562575},
        mapbox_style="carto-positron",
        zoom=9.5,
    )
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choroplethmapbox"))
    fig.update_traces(showlegend=False)
    fig.update_geos(
        projection_type="equirectangular",
        lataxis_range=[miny - 0.1, maxy + 0.1],
        lonaxis_range=[minx - 0.1, maxx + 0.1],
        showlakes=False,
        showcountries=False,
        showframe=False,
        resolution=50,
    )

    size = len(result.to_json().encode("utf-8"))
    fig.update_layout(
        height=450,
        width=700,
        margin={"r": 0, "t": 50, "l": 0, "b": 0},
        title_text=f"Toposimplify value: {epsilon} ({size / 1000} KB)",
    )

    fig.show(renderer="png")  # replace with fig.show() to allow interactivity

## Regionalize points

How to return original regions without clipping and select them using list of points. Showed using list of metro stations in Paris.

Here `admin_level` equal to 8 defines communes in France.

In [ ]:
import requests

r = requests.get("https://raw.githubusercontent.com/w8r/paris-metro-graph/master/metro.json").json()
stations_gdf = gpd.GeoDataFrame(
    {"geometry": [Point(s["longitude"], s["latitude"]) for s in r["nodes"]]}, crs="EPSG:4326"
)
stations_gdf

In [ ]:
abr = AdministrativeBoundaryRegionalizer(
    admin_level=8, return_empty_region=False, clip_regions=False
)
paris_districts_result = abr.transform(gdf=stations_gdf)

In [ ]:
paris_districts_result.head()

In [ ]:
folium_map = plot_regions(paris_districts_result, tiles="CartoDB positron")
stations_gdf.explore(
    m=folium_map,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=1),
)